<a href="https://colab.research.google.com/github/zeeshansalim1234/PURE-Research/blob/main/Pachira_nan_reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing & Importing the required libraries


In [ ]:

!pip install sentence-transformers
!pip install tweepy
!pip install bert-extractive-summarizer
!pip install nltk
!pip install google-cloud-vision

     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 2.8 MB 30.4 MB/s 
     |████████████████████████████████| 1.2 MB 38.8 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 32.1 MB/s 
     |████████████████████████████████| 636 kB 51.5 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=e38642665b4a7fd81558c2a8719eddcd27e8752a2212a9c7c973c4ce634ef577
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 369 kB 5.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 


In [ ]:
import pandas as pd
import json
import os
from sentence_transformers import SentenceTransformer, util
from google.colab import drive
import tweepy
import re
import os,io
from google.cloud import vision
from google.cloud.vision_v1 import types
import pandas as pd
import nltk

ACCESS_TOKEN="1401579178072231936-98mY1wOw1UR3GHOjdo4ePnfdfUHt6n"
ACESS_TOKEN_SECRET="QiIslicgx8GNGrGDPrx8xNZc2WoCyrUsZumgXSyGzGLOx"
CONSUMER_KEY="uaiStuv7EYdYxWSomPKHvSSF5"
CONSUMER_SECRET="JOQryy37w9HPMSrSw8msyyb048iqeHmK4xCRyWP1oBhLKwLYlb"

auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACESS_TOKEN_SECRET)
api=tweepy.API(auth)

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'seismic-diorama-316110-5569927e0d86.json'
#client = vision.ImageAnnotatorClient()

# Function for pre-processing data (A list of Strings should be passed as the argument)

In [ ]:
def pre_process(tweets):

    for i in range(0, len(tweets)):

        if (tweets[i] is not None):

            if(tweets[i]!=tweets[i]):
                tweets[i]=""

            tweets[i] = tweets[i].lower()  # To lower case
            tweets[i] = tweets[i].replace('@','')  # remove @
            tweets[i] = tweets[i].replace('#','')  # remove #
            tweets[i] = remove_urls(tweets[i])  # remove URL
            tweets[i] = remove_emojis(tweets[i])  # remove emojis
            tweets[i] = "".join(j for j in tweets[i] if j not in (
            "?", ".", ";", ":", "!", "-", ",", "[", "]", "(", ")", "’", "‘", '"', "$", "'", "“", "”", "•", "=", "+",
            "%", "/", "&", "|", "~"))  # remove punctuations
            tweets[i] = removeNonEnglishWordsFunct(tweets[i])

    return tweets

def remove_urls (str):    # Removes URL from strings

    str = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str, flags=re.MULTILINE)
    return(str)


def remove_emojis(data):  # Removes Emojis from strings

    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

def removeNonEnglishWordsFunct(x):  # Remove non-rnglish characters from strings

    new_string=re.sub('[^a-zA-Z0-9]',' ',x)

    cleaned_string=re.sub('\s+',' ',new_string)
    return cleaned_string

# Extracting charitable organisation data from Pachira dataset and generating embedding using Specter ML model

In [ ]:
import numpy as np
import glob
li=[]
paths=['/content/charities_bc.csv','/content/charities_ab.csv','/content/charities_qc.csv','/content/charities_sk.csv','/content/charities_pe.csv','/content/charities_yt.csv','/content/charities_on.csv','/content/charities_nu.csv','/content/charities_nt.csv','/content/charities_ns.csv','/content/charities_nl.csv','/content/charities_nb.csv','/content/charities_mb.csv']

for path in paths:

  df = pd.read_csv(path, index_col=None, header=0) #reading from csv file and creating a pandas dataset
  df.columns=['CharityName','URL','City','Province','PostalCode','CharityType','CharityCategory','OngoingPrograms','NewPrograms']
  li.append(df)

df=pd.concat(li, axis=0, ignore_index=True)

df = df[df['OngoingPrograms'].notna()]

charities=df.CharityName.values+" "+df.OngoingPrograms.values # charities list hold the organisation names and description
cities=df.City.values #cities list holds city names

print(len(charities), "Charities")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')   # loading specter model
#To encode the descriptions to a single string
charity_texts = [charity for charity in charities]
cities_texts=[city for city in cities]
"""
#Compute embeddings for all descriptions
corpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)  # generating embeddings using charities name and description"""


77646 Charities


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

'\n#Compute embeddings for all descriptions\ncorpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)  # generating embeddings using charities name and description'

In [ ]:
pip install torchvision

In [ ]:
"""import torch
torch.save(corpus_embeddings, 'charities_tensor.pt')"""

"import torch\ntorch.save(corpus_embeddings, 'charities_tensor.pt')"

In [ ]:
import torch
corpus_embeddings=torch.load('/content/drive/MyDrive/charities_tensor.pt')

In [ ]:
"""import numpy as np
df = pd.read_csv('/content/charities_bc.csv') #reading from csv file and creating a pandas dataset

df = df.dropna() #dropping blank rows

print(df.values[0])

charities=df.CharityName.values+" "+df.OngoingPrograms.values # charities list hold the organisation names and description
cities=df.City.values #cities list holds city names

print(len(charities), "Charities")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')   # loading specter model

#To encode the descriptions to a single string
charity_texts = [charity for charity in charities]
cities_texts=[city for city in cities]

#Compute embeddings for all descriptions
corpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)  # generating embeddings using charities name and description"""


'import numpy as np\ndf = pd.read_csv(\'/content/charities_bc.csv\') #reading from csv file and creating a pandas dataset\n\ndf = df.dropna() #dropping blank rows\n\nprint(df.values[0])\n\ncharities=df.CharityName.values+" "+df.OngoingPrograms.values # charities list hold the organisation names and description\ncities=df.City.values #cities list holds city names\n\nprint(len(charities), "Charities")\n\n#We then load the allenai-specter model with SentenceTransformers\nmodel = SentenceTransformer(\'allenai-specter\')   # loading specter model\n\n#To encode the descriptions to a single string\ncharity_texts = [charity for charity in charities]\ncities_texts=[city for city in cities]\n\n#Compute embeddings for all descriptions\ncorpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)  # generating embeddings using charities name and description'

# Function to find all charities in the same location which is passed as the argument (pass a string as location)

In [ ]:

# Returns all charities at the provided location

def search_papers_location(location):

   results=[]
   for i in range(0,len(cities_texts)):
     if(cities_texts[i]==location):
       results.append(i)
   return results

  

# Randomly generating 100 priority charities (Replace with actual priority charities when provided by Pachira)

In [ ]:
# Generates 100 random charities as the priority

import random
from termcolor import colored

#Generate 5 random numbers between 10 and 30
priority = random.sample(range(0, 1520), 100)
print(priority)


[1376, 76, 159, 420, 81, 342, 462, 711, 1343, 119, 242, 1080, 934, 1210, 1363, 1340, 231, 310, 1344, 723, 1147, 1001, 346, 684, 16, 1277, 143, 568, 1479, 1155, 800, 906, 707, 113, 295, 1112, 887, 937, 1171, 949, 796, 196, 1444, 69, 1162, 1313, 666, 54, 767, 1133, 152, 290, 21, 1014, 947, 1423, 1418, 97, 951, 1403, 739, 523, 353, 1519, 118, 140, 361, 1333, 1041, 1487, 571, 813, 106, 282, 169, 162, 640, 401, 1375, 463, 750, 701, 216, 1508, 517, 1228, 115, 173, 1357, 252, 610, 1090, 1516, 101, 1100, 612, 816, 762, 1308, 302]


# Stores the charities recommendations for the passed string argument in a list (Only for priority charities)

In [ ]:
# For generating recommendations for priority organizations

def generate_priority(title):

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10) # Generates a 2D matrix that contains the id and similarity score of the top 10 matching charities
   search_hits = search_hits[0]
   
   count = 0
   top_header = "\n\nPriority Charities\n\n"
   print(colored(top_header,'yellow'))

   flag=0

   for hit in search_hits:

          for i in range(0,len(priority)):
            if(hit['corpus_id']==priority[i]):   # checks if the charity in top 10 is a part of the priority charities
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>0.70):
            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values

            print("\n"+str(count)+") "+colored(str(k[0][0]), 'red'))
            print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0
            count+=1


# Stores the charities recommendations for the passed string argument in a list (for all charities)

In [ ]:

# For Generating recommendation for all organizations

def search_papers(title,city_indices,status,num_results):

   #status signifies city 0 is city based
   #num_results signifies the number of results we wish to generate

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   flag=0
   tracker=0 #basically counter for the non location segment

   check_numreco_tracker=0  #to track the number of location based reco

   for hit in search_hits:
        
        if(status==0) :

          for i in range(0,len(city_indices)):
            if(hit['corpus_id']==city_indices[i]):
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>=0.80):

            related_charities = charities[hit['corpus_id']]
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values
            

            print("\n"+str(check_numreco_tracker+1)+") "+colored(str(k[0][0]), 'red'))
            all_names.append(str(k[0][0]))
            print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
            all_scores.append(search_hits[count]['score'])
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
            check_numreco_tracker+=1 #to add the number of reco with location
            flag=0

        else:

          if(tracker<num_results): #For normal non-location based (passed blank array in the function call at end) 

            if(len(city_indices)==0 and search_hits[count]['score']>=0.80):

              related_charities = charities[hit['corpus_id']]
              subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms'] == related_charities]
              k=subsetDataFrame.values       
              print("\n"+str(tracker+1)+") "+colored(str(k[0][0]), 'red'))
              all_names.append(str(k[0][0]))
              print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
              all_scores.append(search_hits[count]['score'])
              print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
              all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
              flag=0
              tracker+=1

            elif(len(city_indices)!=0 and cities_texts[hit['corpus_id']]!=cities_texts[city_indices[0]]): #for recursice location based (non-location recos)

              related_charities = charities[hit['corpus_id']]
              subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms'] == related_charities]
              k=subsetDataFrame.values       
              print("\n"+str(6-num_results+tracker)+") "+colored(str(k[0][0]), 'red'))
              all_names.append(str(k[0][0]))
              print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
              all_scores.append(search_hits[count]['score'])
              print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
              all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
              flag=0
              tracker+=1

        count+=1 #keep track of elements in search_hits

   if(status==0): 

     search_papers(title,city_indices,1,5-check_numreco_tracker)  # now sending non-location based reco if didnt reach top 5 already


#search_papers("alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary",[],1,10)

#want to make a big impact donations made through birdies4kids for the alberta diabetes foundation are matched up to 50  birdies for kids runs until august 15th 2021  to donate and read more 

#alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary

#in the news the kidney foundation encourages other canadian organizations and companies to to adopt policies to support living organdonors read more about kidneycanada s wage replacement policy for living organ and tissue donation

#participate in the scotiabank calgary marathon charity challenge to help children amp families at childrens cottage society theres an event for everyone of all abilities register or donate at  thanks for your consideration children families 

#can you help us many children need their very own books strong reading role models amp safe fun reading spaces your donation helps more children experience the magic of reading amp a lifetime of opportunity 

# Using Google PyTrends API to get trending topics

In [ ]:
pip install pytrends


In [ ]:
#df_trends = pytrends.trending_searches(pn='canada')
#df_trends.head()


In [ ]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US') #specify language

"""
data=pytrends.trending_searches('canada')
print(data[0].loc[0])
"""

pytrends.build_payload(kw_list=['protest','charities','events','movements'],geo='CA',timeframe='now 7-d')  # generate the query
related_queries = pytrends.related_queries()
result=list(related_queries.values())
query=list(result[0].values())[1]
#query="Women's soccer final"
print(query)

#search_papers(query,[],1,5)

#list(result[0].values())[1].loc[0].at['query']

                             query  value
0                  winkler protest  83250
1           bolton ontario protest  81950
2                   bolton ontario  81950
3             trucker protest 2021  16100
4                   bolton protest  15000
5          pearson airport protest  13000
6           trudeau surrey protest   6800
7                   trudeau surrey   6800
8   robin cook book crossword clue   6350
9                      tlp protest   5900
10                  rise up durham   5900
11          former atlanta stadium   5900
12           the heavens crossword   5450
13         bay area city crossword   5450
14     courageously crossword clue   5000
15                 trudeau protest   1000
16                         trudeau   1000
17                      bc protest    170
18               anti mask protest    170
19        vaccine passport protest    160
20                vaccine passport    160
21                 ontario protest    160
22     bc vaccine passport protest

# Menu to generate :
#  1. Trending Charities Recommendation
#  2. Priority Charities Recommendation
#  3. General Chairities Recommendation

#  for each individual sentence provided from a conversation.

In [ ]:
all_names=[]
all_descriptions=[]
all_scores=[]

print("What basis do you want to generate a recommendation on ? ")
print("\n1. Enter a sentence  ")
print("2. Trending tweet on a particular hashtag ")
choice=input("Enter your choice : ")

if(choice=="1"):

  title=input("\nEnter a sentence : ")
  enable_loc=str(input("Do you wish to enable location ? "))
  if(enable_loc=="yes"):
    location=str(input("Enter location : "))

  pytrends = TrendReq(hl='en-US') #specify language
  pytrends.build_payload(kw_list=['protest','cahrities','events','movements'],geo='CA',timeframe="now 7-d")  # generate the query
  related_queries = pytrends.related_queries()
  result=list(related_queries.values())
  query=list(result[0].values())[1].loc[1].at['query']
  print("\n"+query)
  
  top_header = "\n\nPopular in your Region\n\n"
  print(colored(top_header,'magenta'))
  search_papers(query,[],1,5)
  

  if(enable_loc=="yes"):

    status=0
    loc_indices=search_papers_location(location)
    generate_priority(title)
    top_header = "\n\nTop related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,loc_indices,status,10)

  else:

    
    temp=[]
    status=1
    generate_priority(title)
    top_header = "\n\nTop related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,temp,status,10)

elif(choice=="2"):

  tag = input("\nEnter a hashtag : ")
  tweets=tweepy.Cursor(api.search,q=tag,result_type='popular',tweet_mode="extended").items(1)
  temp=[]
  for tweet in tweets :

      temp.append(tweet.full_text)

  temp=pre_process(temp)
  tweet=temp[0]
  print("\nMost famous tweet : \n"+colored(tweet,'orange'))
  search_papers(tweet)


What basis do you want to generate a recommendation on ? 

1. Enter a sentence  
2. Trending tweet on a particular hashtag 
Enter your choice : 1

Enter a sentence : cancer
Do you wish to enable location ? no

bolton ontario protest


Popular in your Region



1) ROBIN CROSBY EVANGELISTIC ASSOCIATION
similiarity score of 0.90
Description of charity : ROBIN CROSBY EVANGELISTIC ASSOCIATION BIBLE STUDY WHITBY

2) DEVON PIONEER 73 CLUB
similiarity score of 0.90
Description of charity : DEVON PIONEER 73 CLUB NONE DEVON

3) LA SAMARITAINE DE MONT-TREMBLANT
similiarity score of 0.89
Description of charity : LA SAMARITAINE DE MONT-TREMBLANT COMPTOIR ALIMENTAIRE ET VESTIMENTAIRE POUR PERSONNES DÉFAVORISÉESFOURNITURE ARTICLE SCOLAIRE AUX ENFANTS ISSUS DE FAMILLE DÉFAVORISÉESDISTRIBUTION DE PANIERS DE NOELAIDE PONCTUELLE AUX PERSONNES DANS LE BESOIN MONT-TREMBLANT

4) CRESTON MINISTERIAL ASSOCIATION
similiarity score of 0.89
Description of charity : CRESTON MINISTERIAL ASSOCIATION BENEVOLENT GIVI

# Extracting artificial conversation history created for a user (**To be replaced with actual user history from Pachira**)

In [ ]:
import pandas as pd
df_history = pd.read_csv('/content/drive/MyDrive/artificial_conversation.csv',sep=";", encoding='mac_roman')

#charities =df.CharityName.values+" "+df.OngoingPrograms.values

print(str(df_history.columns[0]))

conversations=df_history.Sentence.values

print(conversations)



Sentence
['Forest-dependent communities in Laos make almost all of their livelihoods from their forestsí natural resources. Find out how they started seedling nurseries and planting initiatives to restore their essential forests in partnership with WWF-Laos.'
 'NSW recorded 356 new locally acquired cases of COVID-19 in the 24 hours to 8pm last night.'
 "Thank you, everyone! We raised $232 during today's charity punishment stream. Our donation was enough to send 4 girls to school for a year, thanks to the International Rescue Committee! We also were able to gift a few steam gift cards to chat &have a fun day!"
 'The military junta is the root cause of humanitarian crisis in #Myanmar. The junta has cut all supply routes to #Mindat & IDPs are now at risk of food shortage. In July, the junta has prevented #UNHCR \n@Refugees\n from delivering aid to IDPs in #Chin State.'
 'Vegetable producer says it is losing thousands of pounds a week and throwing away food due to labour shortage'
 'Figure

# Storing the recommendations from each conversation in the history dataset to be filtered later

In [ ]:


for i in range(0,len(conversations)):
  print(colored(conversations[i],'magenta'))
  search_papers(conversations[i],[],1,10)
  print("\n\n\n\n\n")



print(all_names)
print(all_scores)

Forest-dependent communities in Laos make almost all of their livelihoods from their forestsí natural resources. Find out how they started seedling nurseries and planting initiatives to restore their essential forests in partnership with WWF-Laos.

1) FORESTS ONTARIO FORETS DE L'ONTARIO
similiarity score of 0.93
Description of charity : FORESTS ONTARIO FORETS DE L'ONTARIO FORESTS ONTARIO IS THE VOICE FOR OUR FORESTS. WORKING TO PROMOTE A FUTURE OF HEALTHY FORESTS SUSTAINING HEALTHY PEOPLE, FORESTS ONTARIO IS COMMITTED TO THE RE-GREENING OF ONTARIO THROUGH TREE PLANTING EFFORTS ON RURAL LANDS AND IN URBAN AREAS, AS WELL AS THE RENEWAL AND STEWARDSHP OF ONTARIO'S FORESTS THROUGH RESTORATION, EDUCATION AND AWARENESS.FORESTS ONTARIO IS A PROVINCIAL RESOURCE AND TRUSTED AUTHORITY FOR THOSE SEEKING TO INVEST IN THE FUTURE OF OUR FORESTS - THROUGH DONATIONS, SPONSORSHIP, VOLUNTEERISM, TREE PLANTING, COMMUNITY AWARENESS EVENTS AND FOREST MANAGEMENT.PROGRAMS INCLUDE RESTORATION INITIATIVES SUCH

In [ ]:
charity_counter=0
temp_scores=[]
indices=[]
top_names=[]
top_descriptions=[]

for i in range(0,len(all_scores)):
  temp_scores.append(all_scores[i])

all_scores = sorted(all_scores, reverse=True) 



# Showing only top 10 recommendations chosen from all the recommendations generated for different conversations on the basis of the highest similarity scores of the recommendations

## **- This approach was taken to mantain diversity in recommendations while having high quality recommendations (highest degree of textual match)**

## **- This approach can be changed if a sutaible alternate method is devised**

In [ ]:


for i in range(0,10):
  for j in range(0,len(temp_scores)):
    if(all_scores[i]==temp_scores[j]):
      indices.append(j)

print(indices)
for i in range(0, 10):
  top_names.append(all_names[indices[i]])
  top_descriptions.append(all_descriptions[indices[i]])




[15, 16, 95, 17, 18, 19, 20, 75, 96, 97]


In [ ]:
counter=0

choice=input("Want to set a location? ")
print("\n")
print(colored("History Based Recommendations \n",'green'))
if(choice=="yes"):
  user_location=input("Enter City: ")
  print("\n")
  for i in range(0,10):
    if(user_location in top_descriptions[i]):
      print(str(counter+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
      print(top_descriptions[i])
      print("\n")
      counter+=1
    
  for i in range(0,10):
    if(user_location not in top_descriptions[i]):
      print(str(counter+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
      print(top_descriptions[i])
      print("\n")
      counter+=1
else:
   for i in range(0,10):
  
    print(str(i+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
    print(top_descriptions[i])
    print("\n")


pytrends = TrendReq(hl='en-US') #specify language
pytrends.build_payload(kw_list=['protest','charities','events','movements'],geo='CA',timeframe="now 7-d")  # generate the query
related_queries = pytrends.related_queries()
result=list(related_queries.values())
query=list(result[0].values())[1].loc[1].at['query']
print("Trending Topic in Canada: ""\n"+colored(query,"magenta"))

top_header = "\n\nPopular in your Region\n\n"
print(colored(top_header,'magenta'))
search_papers(query,[],1,5)
    

Want to set a location? no


History Based Recommendations 

1) FORESTS ONTARIO FORETS DE L'ONTARIO(0.9007053971290588)
FORESTS ONTARIO FORETS DE L'ONTARIO FORESTS ONTARIO IS THE VOICE FOR OUR FORESTS. WORKING TO PROMOTE A FUTURE OF HEALTHY FORESTS SUSTAINING HEALTHY PEOPLE, FORESTS ONTARIO IS COMMITTED TO THE RE-GREENING OF ONTARIO THROUGH TREE PLANTING EFFORTS ON RURAL LANDS AND IN URBAN AREAS, AS WELL AS THE RENEWAL AND STEWARDSHP OF ONTARIO'S FORESTS THROUGH RESTORATION, EDUCATION AND AWARENESS.FORESTS ONTARIO IS A PROVINCIAL RESOURCE AND TRUSTED AUTHORITY FOR THOSE SEEKING TO INVEST IN THE FUTURE OF OUR FORESTS - THROUGH DONATIONS, SPONSORSHIP, VOLUNTEERISM, TREE PLANTING, COMMUNITY AWARENESS EVENTS AND FOREST MANAGEMENT.PROGRAMS INCLUDE RESTORATION INITIATIVES SUCH AS THEIR 50 MILLION TREE PROGRAM, COMMUNITY TREE PLANTING EVENTS AND THE NATIONL PLANTING PROGRAM FOREST RECOVERY CANADA, AS WELL AS EDUCATION PROGRAMS INCLUDING ENVIROTHON, PLANT, AND FORESTRY CONNECTS. TORONTO


2) T